In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sn
import glob
import os
import sys

import utm #pip install utm
import plotly.express as px #pip install plotly

In [3]:
root_dir = os.path.dirname(os.getcwd())

# Directorios principales
ProcesamientoDatos_dir = os.path.join(root_dir, 'a. Procesamiento de datos')
Estudios_dir = os.path.join(root_dir, 'b. Estudio Correlacion y Sistema')
Simulacion_dir = os.path.join(root_dir, 'c. Simulacion sistema')

# Datos de entrada
Datos_dir = os.path.join(os.path.dirname(root_dir), '1. Datos de entrada')
GenData_dir = os.path.join(Datos_dir, 'Generacion')
datosCEN_dir = os.path.join(GenData_dir, 'Datos_CEN')

# Directorios de procesamiento de datos
Output_dir = os.path.join(ProcesamientoDatos_dir, 'Output')

#Crear las carpetas que no existan
if not os.path.exists(Output_dir): os.makedirs(Output_dir)

# Leer datos de reporte centrales, extraido de Infotecnica
reporte_file = os.path.join(GenData_dir, 'reporte_centrales.xlsx')
df_reporte_raw = pd.read_excel(reporte_file, sheet_name='Centrales', skiprows=6)

In [4]:
df_reporte = df_reporte_raw.loc[:,['ID','Nombre','11.1.39 Coordenada este','11.1.39 Coordenada norte','11.1.39 Zona o huso [ej: 18H, 19J]','11.1.4 Potencia máxima bruta, para cada tipo de combustible que pueda operar']]
df_reporte.columns = ['ID','Nombre','Este','Norte','Zona','PotenciaMax']

# Agregar columna de latitud y longitud a df_reporte y llenarla con 0.0
df_reporte['Latitud'] = 0.0
df_reporte['Longitud'] = 0.0

for idx,row in df_reporte.iterrows():
    Nombre = row['Nombre']
    Este = str(row['Este'])
    Norte = str(row['Norte'])
    ZonaHuso = str(row['Zona'])

    Este = Este.strip().replace(',','.').split(' ')[0]
    Norte = Norte.strip().replace(',','.').split(' ')[0]

    # Contar cuantos . hay en Norte y Este, si hay mas de 1, se deja el ultimo
    if Este.count('.') > 1:
        Este_0,Este_1 = Este.split('.')[:-1],Este.split('.')[-1:]
        Este = ''.join(Este_0) + '.' + ''.join(Este_1)
    if Norte.count('.') > 1:
        Norte_0,Norte_1 = Norte.split('.')[:-1],Norte.split('.')[-1:]
        Norte = ''.join(Norte_0) + '.' + ''.join(Norte_1)

    ZonaHuso = ZonaHuso.strip()
    if len(ZonaHuso) == 3:
        Zona,Huso = ZonaHuso[:2],ZonaHuso[2:].strip()
    elif len(ZonaHuso) == 2:
        Zona = ZonaHuso
        Huso = 'H'

    if Huso == 'S':
        Huso = 'J'

    ZonaHuso = str(Zona) + str(Huso)

    Este = float(Este)
    Norte = float(Norte)

    try:
        x_lat,y_lon = utm.to_latlon(Este,Norte,int(Zona),Huso)

        # Revisar si la central esta dentro de Chile, si no, cambiar el huso
        lim_Latitud_1 = [-34, -43]
        lim_Longitud_1 = [-65, -68]

        lim_Latitud_2 = [-20, -40]
        lim_Longitud_2 = [-74.5, -79]


        if (((-lim_Latitud_1[0] < -x_lat) and (-x_lat < -lim_Latitud_1[1])) and ((-lim_Longitud_1[0] < -y_lon) and (-y_lon < -lim_Longitud_1[1]))):
            Zona = 18
            x_lat,y_lon = utm.to_latlon(Este,Norte,int(Zona),Huso)
        elif (((-lim_Latitud_2[0] < -x_lat) and (-x_lat < -lim_Latitud_2[1])) and ((-lim_Longitud_2[0] < -y_lon) and (-y_lon < -lim_Longitud_2[1]))):
            Zona = 19
            x_lat,y_lon = utm.to_latlon(Este,Norte,int(Zona),Huso)


        df_reporte.loc[idx,'Latitud'] = x_lat
        df_reporte.loc[idx,'Longitud'] = y_lon
        df_reporte.loc[idx,'Zona'] = ZonaHuso
    except:
        print('Error en el punto: ',Nombre)
        print('  Este: ',Este)
        print('  Norte: ',Norte)
        print('  Zona: ',Zona)
        print('  Huso: ',Huso)
        # Quitar las centrales que no tienen coordenadas
        df_reporte.drop(idx,inplace=True)

df_reporte.reset_index(drop=True,inplace=True)

Error en el punto:  TER TRES PUENTES (SM)
  Este:  72901.2
  Norte:  14847.6
  Zona:  19
  Huso:  F
Error en el punto:  PE CABO NEGRO (SM)
  Este:  nan
  Norte:  nan
  Zona:  na
  Huso:  n
Error en el punto:  PMGD PFV LUNA
  Este:  6211449.0
  Norte:  278119.0
  Zona:  19
  Huso:  H
Error en el punto:  PMGD PFV QUELTEHUE SOLAR
  Este:  6193158.82
  Norte:  320048.89
  Zona:  19
  Huso:  H
Error en el punto:  PMGD PFV PITOTOY
  Este:  6193061.0
  Norte:  319927.0
  Zona:  19
  Huso:  H
Error en el punto:  PMGD PFV SAN RAMIRO
  Este:  278258.06
  Norte:  62345212.25
  Zona:  19
  Huso:  H
Error en el punto:  PFV DEL DESIERTO
  Este:  7343977.3
  Norte:  541029.48
  Zona:  19
  Huso:  J
Error en el punto:  PMGD PFV LAS TENCAS
  Este:  307.644
  Norte:  6189.377
  Zona:  19
  Huso:  h
Error en el punto:  PMGD PFV MALINKE
  Este:  288.155
  Norte:  6260.478
  Zona:  19
  Huso:  H
Error en el punto:  PE VIENTOS PATAGONICOS (SM)
  Este:  376.863
  Norte:  4132.099
  Zona:  19
  Huso:  F


In [5]:
# Reemplazar las coordenadas de PE MESAMAVIDA por -37.490163,-72.4620369
idx = df_reporte[df_reporte['Nombre'] == 'PE MESAMAVIDA'].index[0]
df_reporte.loc[idx,'Latitud'] = -37.490163
df_reporte.loc[idx,'Longitud'] = -72.4620369

In [6]:
df_reporte

,ID,Nombre,Este,Norte,Zona,PotenciaMax,Latitud,Longitud
0,2,TER MAULE,735611.986566,6083902.072249,18H,6.099,-35.360399,-72.406870
1,3,TER CHILOE,612679.972693,5228107.067444,18G,9.227,-43.091383,-73.615521
2,4,TER CONSTITUCION,735611.986566,6083902.072249,18H,9.593,-35.360399,-72.406870
3,5,HP MARIPOSAS,294883,6058490,18H,6.3,-35.596088,-71.264169
4,6,HP EL RINCON,356744.76,6286886.3,19H,0.289,-33.548059,-70.543060
...,...,...,...,...,...,...,...,...
897,2142,PMGD TER EL JARDIN,644510,5392638,18G,3.01,-41.604895,-73.265800
898,2146,PMGD PFV QUEMADOS,343913.43,6220841.42,19H,7.01,-34.141669,-70.692927
899,2148,PMGD PFV PSF EL PERAL,343057,6264451,19H,9.01,-33.748399,-70.694402
900,2152,PMGD PFV EL CHERCAN,288425,6126788,19H,9.015,-34.979394,-71.317789


In [7]:
df_reporte.to_excel(os.path.join(Output_dir,'Coordenadas_CEN.xlsx'),index=False,sheet_name='Coordenadas')

# Visualizacion rapida de centrales
---

In [8]:
fig = px.scatter_mapbox(df_reporte, 
                        lat="Latitud", lon="Longitud", 
                        hover_name="Nombre",
                        hover_data=["ID","Este","Norte","Zona"],
                        zoom=4,
                        mapbox_style='open-street-map')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()